# From ChiP to Core Regulatory Circuitry.
Jérémie Kalfon
@jkobject

[www.jkobject.com](https://www.jkobject.com)

in partnership with the Broad Institute

thanks to young Lab @ MIT

In [12]:
from Bio import SeqIO
import os
path_to_rose = "/Users/jeremie/Documents/logiciels/rose"
os.chdir(path_to_rose)
import ROSE_utils
os.chdir(a)

ImportError: No module named Bio

### you might need to install an extention to curl to download from an sftp server. otherwise reinstall the latest version of curl with libssh

on mac: 
`brew install curl --with-libssh2=/usr/local`
after installing [conda]( 
```
conda config --add channels defaults
conda config --add channels bioconda
conda config --add channels conda-forge
```
then
`conda config install bowtie2 && pip install MACS2`
then `git clone` and provide path to [ROSE](https://bitbucket.org/young_computation/rose/src) and [CRCMapper](https://bitbucket.org/young_computation/crcmapper)

for CRC mapper you need [meme](http://meme-suite.org/doc/install.html?man_type=web)

In [49]:
mypath= "datatosend"
! curl -k "sftp://bioinfodata.wistar.upenn.edu/datatosend.tar.gz" --user [NAME] > $mypath.tar.gz


7567143977

## the call might stop for many reasons. in this case just run this function below to continue it from where you started as the -R option is not working well with curl and sftp

In [ ]:
def notfinished():
    s = os.path.getsize(mypath+'.tar.gz')
    ! curl -C $s -k "sftp://bioinfodata.wistar.upenn.edu/datatosend.tar.gz" --user [NAME] > $mypath1.tar.gz
    ! cat $mypath.tar.gz $mypath1.tar.gz > $mypath2.tar.gz
    ! rm $mypath.tar.gz $mypath1.tar.gz
    ! mv $mypath2.tar.gz $mypath.tar.gz

### here we prepare the libs and the alignments with the hg19/37 version of the genome as it is the best one currently working with CRCmapper

In [ ]:
! gzip -d $mypath.tar.gz
! curl ftp://ftp.ensembl.org/pub/grch37/current/fasta/homo_sapiens/dna/Homo_sapiens.GRCh37.dna.toplevel.fa.gz > reference.fa.gz
! gzip -d reference.fa.gz
! mkdir reference
os.chdir("reference")
! wget --timestamping 'ftp://hgdownload.cse.ucsc.edu/goldenPath/hg19/chromosomes/chr{1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,X,Y}.fa.gz'
! gzip -d *.fa.gz

### some visualization

In [12]:
for i, rec in enumerate(SeqIO.parse(mypath+"/54-mp457-PDX34-H3K27ac-r2_S7_R1.fastq", "fastq")):
    print(rec)
    if i > 10:
        break    

ID: 7001405:985:HJ7YWBCX2:1:1102:1233:1928
Name: 7001405:985:HJ7YWBCX2:1:1102:1233:1928
Description: 7001405:985:HJ7YWBCX2:1:1102:1233:1928 1:N:0:GCCAATGT
Number of features: 0
Per letter annotation for: phred_quality
Seq('NTTCCAATGATCTCACATTAAACTGAAGCCTCTTTCCTCTGACAAACGTTTCCG...GAN', SingleLetterAlphabet())
ID: 7001405:985:HJ7YWBCX2:1:1102:1408:1913
Name: 7001405:985:HJ7YWBCX2:1:1102:1408:1913
Description: 7001405:985:HJ7YWBCX2:1:1102:1408:1913 1:N:0:GCCAATGT
Number of features: 0
Per letter annotation for: phred_quality
Seq('NCTAATGTTTTTCTATTTTTAATAGAGATGTGGTTTCACCATGTTGGTCAGGCT...CCC', SingleLetterAlphabet())
ID: 7001405:985:HJ7YWBCX2:1:1102:1276:1938
Name: 7001405:985:HJ7YWBCX2:1:1102:1276:1938
Description: 7001405:985:HJ7YWBCX2:1:1102:1276:1938 1:N:0:GCCAATGT
Number of features: 0
Per letter annotation for: phred_quality
Seq('NAGTCACGTGACCCAGAAACTTGTCCAAGGAGGCATGCATCTCTGGGCTGAAGT...GAN', SingleLetterAlphabet())
ID: 7001405:985:HJ7YWBCX2:1:1102:1494:1973
Name: 7001405:985:HJ7YWBCX2

## here we can see the -logp that the base call was incorrect (called phred quality/sanger quality...) 

In [30]:
for i, rec in enumerate(SeqIO.parse(mypath+"/54-mp457-PDX34-H3K27ac-r2_S7_R1.fastq", "fastq")):
    print(rec.seq)
    if i > 10:
        break

AAGTANCTCTGCTTAGGAATGAGAATGGTATTTGACTTGGTTTTAANGNATTAAANAGGTTTTTCTACCANCANA
AAATGNACTGCTTGTTCTACACCACTTATGTTTTTTTTTATAGTGANCNTTATAANCTAAGAGAAACAGCNGANT
GAAGTNGATTTTGCTTGTATATTTTGAGAGTAGGAGTAGCTTTGCTNCNTAAAACNCATACCTAAATCCTNCAGA
GTTGTNTTAAGTGAAAGAGATAATGCATATAACAGTAGTGACTATANTNATTGAGCACTTACTATGTGACNTGCA
TATGCNCAAGTATGTGTGTGTGCTAGCCAATGTGCAGAGAAGCTTTNCNTCCTAAATGGGACATCTTCATNAAAT
ATGCTNTGAAGAAGCAAGCTGCAGTGTTGGAGAGGCCCATGTGGCANGGAAATGAGGGTGGCCTCCAGTCAATGG
CTCACNGTGTTACCAGGTTGGAATGCAGTGGCGCGATCTCGGCTCACTGCAACCTCTGCCTCTTGGGTTCAAGTA
AGTTTNTGGCTTTTACTACAATATATGTAGAACTGGATATTTAGAATATCATTAAAACAAAGTACTTTGTGATTC
TAGGANTTGTTTGCATGTGGAGTCAGATTCTCTGAGAACAAATGCCCATAATTTTGTGTATTTGGGCATAAAACA
CTACTNTGAAATTTTGTATAGATAGGGTTAAATTTAGGAATATTTTCAAGACTTTGAATTACATAGAGTATTATC
GTGCCNATTGGTGTATTTACAATCCCCGAGCTAGACACAAAGGTTCTCCAAGGCCCCACCAGACTCAGGAGCCCA
TGTGANGTGTTCGTTCATCTCACAGAGTTTAACCTTTCTTTTCATAGAGCAGTTAGGAAACAGTCTGTTTGTAAA
CAAAANAATGGAATCCAAAATCATTGACTGGAAAGGCTGGGTGTCGAAAGGAATTGACTCCAATGGAATGGAATC
TGTCTNTCACAA

In [6]:
! MACS2 callpeak -h

usage: MACS2 callpeak [-h] -t TFILE [TFILE ...] [-c [CFILE [CFILE ...]]]
                      [-f {AUTO,BAM,SAM,BED,ELAND,ELANDMULTI,ELANDEXPORT,BOWTIE,BAMPE,BEDPE}]
                      [-g GSIZE] [-s TSIZE] [--keep-dup KEEPDUPLICATES]
                      [--outdir OUTDIR] [-n NAME] [-B] [--verbose VERBOSE]
                      [--trackline] [--SPMR] [--nomodel] [--shift SHIFT]
                      [--extsize EXTSIZE] [--bw BW] [-m MFOLD MFOLD]
                      [--fix-bimodal] [-q QVALUE | -p PVALUE]
                      [--scale-to {large,small}] [--ratio RATIO]
                      [--down-sample] [--seed SEED] [--tempdir TEMPDIR]
                      [--nolambda] [--slocal SMALLLOCAL] [--llocal LARGELOCAL]
                      [--max-gap MAXGAP] [--min-length MINLEN] [--broad]
                      [--broad-cutoff BROADCUTOFF] [--cutoff-analysis]
                      [--call-summits] [--fe-cutoff FECUTOFF]
                      [--buffer-size BUFFER_SIZE] [--to-larg

In [37]:
! bowtie2 -h

Bowtie 2 version 2.3.4.3 by Ben Langmead (langmea@cs.jhu.edu, www.cs.jhu.edu/~langmea)
Usage: 
  bowtie2 [options]* -x <bt2-idx> {-1 <m1> -2 <m2> | -U <r> | --interleaved <i>} [-S <sam>]

  <bt2-idx>  Index filename prefix (minus trailing .X.bt2).
             NOTE: Bowtie 1 and Bowtie 2 indexes are not compatible.
  <m1>       Files with #1 mates, paired with files in <m2>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <m2>       Files with #2 mates, paired with files in <m1>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <r>        Files with unpaired reads.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <i>        Files with interleaved paired-end FASTQ reads
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <sam>      File for SAM output (default: stdout)

  <m1>, <m2>, <r> can be comma-separated lists (no whitespace) and can be
  specified many times. 

### Let's build the indexes with bowtie2  

In [68]:
! bowtie2-build $mypath/reference.fa index --threads 8
#maybe try to output bampe files next time (and with higher sensibility) for macs2 to read the paired end information

Settings:
  Output files: "index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 32
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  datatosend/reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:29
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:18
bmax according to bmaxDivN setting: 89524538
Using parameters --bmax 67143404 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 67143404 --dcv 1024
Constructing suffi

### we then produce the alignments

here we use the paired end alignments

In [70]:
! bowtie2 -x index -1 $mypath/54-mp457-PDX34-H3K27ac-r2_S7_R1.fastq.gz -2 $mypath/54-mp457-PDX34-H3K27ac-r2_S7_R2.fastq.gz -S $mypath/mapped.sam --threads 8 -t

Time loading reference: 00:00:00
Time loading forward index: 00:00:01
Time loading mirror index: 00:00:03
Multiseed full-index search: 01:42:07
29050599 reads; of these:
  29050599 (100.00%) were paired; of these:
    8460207 (29.12%) aligned concordantly 0 times
    18598191 (64.02%) aligned concordantly exactly 1 time
    1992201 (6.86%) aligned concordantly >1 times
    ----
    8460207 pairs aligned concordantly 0 times; of these:
      905301 (10.70%) aligned discordantly 1 time
    ----
    7554906 pairs aligned 0 times concordantly or discordantly; of these:
      15109812 mates make up the pairs; of these:
        14345991 (94.94%) aligned 0 times
        452282 (2.99%) aligned exactly 1 time
        311539 (2.06%) aligned >1 times
75.31% overall alignment rate
Time searching: 01:42:12
Overall time: 01:42:12


### we can then use macs2 for the peak calling 

In [15]:
! MACS2 callpeak -f BAMPE -t $mypath/mapped.sam --outdir "peak_called_bampe"
# 

/bin/sh: MACS2: command not found


### Some visualization...

In [72]:
! Rscript peak_called/NA_model.r
from IPython.display import IFrame
IFrame("./peak_called/NA_model.pdf", width=600, height=300)

null device 
          1 


### finally we need some file transformation for rose to be able to process everything well

we transform the outputed bed files into gff for rose to be able to process

In [73]:
import ROSE_utils
ROSE_utils.bedToGFF(bed="/Users/jeremie/Documents/Projects/BroadInstitute/peak_called/NA_summits.bed",output="/Users/jeremie/Documents/Projects/BroadInstitute/peak_called/NA_summits.gff")

we then sort and index the mapped reads

In [74]:
! samtools sort --threads 6 $mypath/mapped.sam > $mypath/mapped_sorted.bam 

[bam_sort_core] merging from 18 files and 6 in-memory blocks...


we erase the now useless file

In [75]:
! rm $mypath/mapped.sam

we sort the reads

In [76]:
! samtools index -@ 7 $mypath/mapped_sorted.bam

### finally we have to switch folders and then call rose
rose will now stitch the enhancers to find the superenhancers in this cell line 

In [77]:
import os
a = ['/Users/jeremie/Documents/Projects/BroadInstitute'] #! pwd
a = a[0]
path_to_rose = "/Users/jeremie/Documents/logiciels/rose"
os.chdir(path_to_rose)
! python ROSE_main.py -g hg19 -i $a/peak_called/NA_summits.gff -r $a/$mypath/mapped_sorted.bam -o $a/superenhancers

folder /Users/jeremie/Documents/Projects/BroadInstitute/superenhancers/ does not exist
folder /Users/jeremie/Documents/Projects/BroadInstitute/superenhancers/gff/ does not exist
folder /Users/jeremie/Documents/Projects/BroadInstitute/superenhancers/mappedGFF/ does not exist
USING /Users/jeremie/Documents/Projects/BroadInstitute/peak_called/NA_summits.gff AS THE INPUT GFF
USING hg19 AS THE GENOME
MAKING START DICT
LOADING IN GFF REGIONS
CHECKING INPUT TO MAKE SURE EACH REGION HAS A UNIQUE IDENTIFIER
REFERENCE COLLECTION PASSES QC
STITCHING REGIONS TOGETHER
PERFORMING REGION STITCHING
MAKING GFF FROM STITCHED COLLECTION
WRITING STITCHED GFF TO DISK AS /Users/jeremie/Documents/Projects/BroadInstitute/superenhancers/gff/NA_summits_12KB_STITCHED.gff
OUTPUT WILL BE WRITTEN TO  /Users/jeremie/Documents/Projects/BroadInstitute/superenhancers/NA_summits_12KB_STITCHED_ENHANCER_REGION_MAP.txt
python ROSE_bamToGFF.py -f 1 -e 200 -r -m 1 -b /Users/jeremie/Documents/Projects/BroadInstitute/datatosen

### We can now create a map of the core regulatory circuitry of this cell line!

they were two lines to change in crcmapper to make it work...
```python
def createExpressionDict(annotationFile, projectFolder, projectName, refseqToNameDict, expressionTable):
```
line 98

and 
```python
for line in expressionTable[1:]:
    trid = line[0]
    geneName = refseqToNameDict[trid]
    if len(expressionTable[1]) == 3:  # when expressionTable is an output from bamToGFF.py
        exp = float(line[2])
    else:  # when expressionTable is passed as an option (2 columns)
        exp = float(line[1])
```
line 120 

creating syntax error

In [22]:
path_to_crcmapper = "/Users/jeremie/Documents/logiciels/crcmapper"
os.chdir(path_to_crcmapper)

! python CRCmapper.py -e $a/superenhancers/NA_summits_AllEnhancers.table.txt -b $a/$mypath/mapped_sorted.bam -g HG19 -f $a/$mypath/reference -s $a/peak_called/NA_summits.bed -n CRC -o $a/crcmapper

{'enhancers': '/Users/jeremie/Documents/Projects/BroadInstitute/superenhancers/NA_summits_AllEnhancers.table.txt', 'name': 'CRC', 'extension': 500, 'expCutoff': 33, 'subpeaks': '/Users/jeremie/Documents/Projects/BroadInstitute/peak_called/NA_summits.bed', 'Enumber': 'supers', 'genome': 'HG19', 'activity': None, 'output': '/Users/jeremie/Documents/Projects/BroadInstitute/crcmapper', 'fasta': '/Users/jeremie/Documents/Projects/BroadInstitute/datatosend/reference.fa', 'bam': '/Users/jeremie/Documents/Projects/BroadInstitute/datatosend/mapped_sorted.bam'}
IDENTIFY EXPRESSED GENES
{'unique': 0, 'matrix': '1', 'extension': '200', 'floor': '0', 'input': '/Users/jeremie/Documents/Projects/BroadInstitute/crcmapperCRC_TSS.gff', 'density': True, 'cluster': None, 'bam': '/Users/jeremie/Documents/Projects/BroadInstitute/datatosend/mapped_sorted.bam', 'sense': 'both', 'output': '/Users/jeremie/Documents/Projects/BroadInstitute/crcmappermatrix.gff', 'total': False, 'rpm': True, 'jxn': False}
[]
mappi

## an automated version

In [ ]:
mypath = "datatosend"
refs = [30,31,32,33,34]
files = {}
path_to_rose = "/Users/jeremie/Documents/logiciels/rose"
path_to_crcmapper = "/Users/jeremie/Documents/logiciels/crcmapper"
a = ! pwd
a = a[0]
os.chdir(path_to_rose)
import ROSE_utils
os.chdir(a)
for ref in refs
    for r in os.listdir(mypath):
        if r[:2] in refs:
            lane = "R1" if "_R1" in r else "R2"
            files[lane] = r
            ! bowtie2 -x index -1 $mypath/$files["R1"] -2 $mypath/$files["R2"] -S $mypath/mapped.sam --threads 8 -t
            ! MACS2 callpeak -t $mypath/mapped.sam --outdir peak_called_$ref
            ! Rscript peak_called_$ref/NA_model.r
            ! samtools sort --threads 6 $mypath/mapped.sam > $mypath/mapped_sorted_$ref.bam 
            ! rm $mypath/mapped.sam
            ! samtools index -@ 7 $mypath/mapped_sorted_$ref.bam 
            os.chdir(path_to_rose)
            ROSE_utils.bedToGFF(bed=a + "/peak_called_"+ref+"/NA_summits.bed",output=a + "/peak_called_"+ ref +"/NA_summits.gff")
            ! python ROSE_main.py -g hg19 -i $a/peak_called_$ref/NA_summits.gff -r $a/$mypath/mapped_sorted_$ref.bam -o $a/superenhancers_$ref
            os.chdir(path_to_crcmapper)
            ! python CRCmapper.py -e $a/superenhancers_$ref/NA_summits_AllEnhancers.table.txt -b $a/$mypath/mapped_sorted_$ref.bam -g hg19 -f $a/$mypath/reference  -s $a/peak_called_$ref/NA_summits.bed -n CRC -o $a/crcmapper_$ref
            os.chdir(a)

In [35]:
def bed_to_gff(input_name, output_name): # thanks to glaaxy
    skipped_lines = 0
    first_skipped_line = 0
    out = open( output_name, 'w' )
    out.write( "##gff-version 2\n" )
    out.write( "##bed_to_gff_converter.py\n\n" )
    i = 0
    for i, line in enumerate( file( input_name ) ):
        complete_bed = False
        line = line.rstrip( '\r\n' )
        if line and not line.startswith( '#' ) and not line.startswith( 'track' ) and not line.startswith( 'browser' ):
            try:
                elems = line.split( '\t' )
                if len( elems ) == 12:
                    complete_bed = True
                chrom = elems[0]
                if complete_bed:
                    feature = "mRNA"
                else:
                    try:
                        feature = elems[3]
                    except:
                        feature = 'feature%d' % ( i + 1 )
                start = int( elems[1] ) + 1
                end = int( elems[2] )
                try:
                    score = elems[4]
                except:
                    score = '0'
                try:
                    strand = elems[5]
                except:
                    strand = '+'
                try:
                    group = elems[3]
                except:
                    group = 'group%d' % ( i + 1 )
                if complete_bed:
                    out.write( 'chr%s\t%s\t\t%d\t%d\t\t\t%s\t.\t\t%s %s;\n' % ( chrom, feature, start, end, score, strand, feature, group  ) )
                else:
                    out.write( 'chr%s\t%s\t\t%d\t%d\t%s\t%s\t.\t\t%s;\n' % ( chrom, feature, start, end, score, strand, group  ) )
                if complete_bed:
                    # We have all the info necessary to annotate exons for genes and mRNAs
                    block_count = int( elems[9] )
                    block_sizes = elems[10].split( ',' )
                    block_starts = elems[11].split( ',' )
                    for j in range( block_count ):
                        exon_start = int( start ) + int( block_starts[j] )
                        exon_end = exon_start + int( block_sizes[j] ) - 1
                        out.write( '%s\tbed2gff\texon\t%d\t%d\t%s\t%s\t.\texon %s;\n' % ( chrom, exon_start, exon_end, score, strand, group ) )
            except:
                skipped_lines += 1
                if not first_skipped_line:
                    first_skipped_line = i + 1
        else:
            skipped_lines += 1
            if not first_skipped_line:
                first_skipped_line = i + 1
    out.close()
    info_msg = "%i lines converted to GFF version 2.  " % ( i + 1 - skipped_lines )
    if skipped_lines > 0:
        info_msg += "Skipped %d blank/comment/invalid lines starting with line #%d." %( skipped_lines, first_skipped_line )
    print info_msg